In [3]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [4]:
path = r"C:\Users\anish\02-2025 Instacart Basket Analysis"

In [5]:
path

'C:\\Users\\anish\\02-2025 Instacart Basket Analysis'

In [6]:
df_customers = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers.csv'))

## Question 4: Wrangle Data

In [8]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
df_customers.shape

(206209, 10)

In [10]:
df_customers = df_customers.rename(columns={ 
    'First Name' : 'first_name',
    'Surnam' : 'last_name', 
    'Gender' : 'gender',
    'STATE' : 'state', 
    'Age' : 'age',
    'date_joined' : 'date_joined', 
    'n_dependants' : 'number_of_dependants', 
    'fam_status' : 'marital_status'})

Renamed the columns and kept the lowercase format.

In [12]:
df_customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## Question 5: Fundamental data quality and consistency checks

In [14]:
df_customers.isnull().sum()

user_id                     0
first_name              11259
last_name                   0
gender                      0
state                       0
age                         0
date_joined                 0
number_of_dependants        0
marital_status              0
income                      0
dtype: int64

There are 11259 null or missing values, but we don't need any further action since it's not vital for the analysis

In [16]:
df_dups = df_customers[df_customers.duplicated()]

In [17]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,number_of_dependants,marital_status,income


There are no duplicate values

In [19]:
df_customers.dtypes

user_id                  int64
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

In [20]:
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

first_name


We can see that first_name is a mixed data type so I will take further action to fix it.

In [22]:
df_customers['first_name'] = df_customers['first_name'].astype('str')

In [23]:
df_customers.dtypes

user_id                  int64
first_name              object
last_name               object
gender                  object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

In [24]:
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

first_name has been fixed

## Question 6: Combine your customer data with the rest of your prepared Instacart data.

In [27]:
df_ords_prods_merge =pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_aggregated.pkl'))

In [28]:
df_ords_prods_merge.dtypes

order_id                         int64
user_id                          int64
order_number                     int64
orders_day_of_week               int64
orders_hour                      int64
days_since_prior_order         float64
product_id                       int64
add_to_cart_order                int64
reordered                        int64
_merge                        category
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
match                         category
price_range_loc                 object
busiest_day                     object
busiest_days                    object
busiest_period_of_day           object
max_order                        int64
loyalty_flag                    object
avg_product_price              float64
spending_flag                   object
median_days_between_orders     float64
order_frequency_flag            object
dtype: object

In [29]:
df_ords_prods_merge = df_ords_prods_merge.drop(columns=['_merge'])

merge column not needed anymore

In [31]:
df_ords_prods_customers_merge = df_ords_prods_merge.merge(df_customers, on = 'user_id', indicator = True)

In [54]:
df_ords_prods_customers_merge.to_pickle(os.path.join(path, 'Data','Prepared Data', 'orders_products_customers_aggregated.pkl'))